In [1]:
import torch
import torch.nn as nn
import pickle
from sklearn.metrics import accuracy_score
import numpy as np

In [4]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [2]:
# load in train and test sets

with open('data/train_fcnn', 'rb') as handle:
    train = pickle.load(handle)
with open('data/test_fcnn', 'rb') as handle:
    test = pickle.load(handle)

In [13]:
class NeuralNet(nn.Module):
    def __init__(self, emb_size):
        super(NeuralNet, self).__init__()
        self.emb_size = emb_size
        self.fc1 = nn.Linear(self.emb_size, self.emb_size * 5)
        self.fc2 = nn.Linear(self.emb_size * 5, int(self.emb_size/2))
        self.fc3 = nn.Linear(int(self.emb_size/2), 20)
        self.fc4 = nn.Linear(20, 1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.1)
        self.layernorm1 = nn.LayerNorm((self.emb_size * 5))
        self.layernorm2 = nn.LayerNorm((20))
    

    def forward(self, x):

        x = self.fc1(x)
        x = self.layernorm1(self.dropout(self.tanh(x)))
        x = self.fc2(x)
        x = self.tanh(x)
        x = self.fc3(x)
        x = self.layernorm2(self.tanh(x))
        x = self.fc4(x)
        x = self.sigmoid(x)

        return x

In [14]:
# train NN

torch.manual_seed(200206323)
emb_size = train.shape[-1] - 1
max_epochs = 30
epoch_print_num = 5
batch_size = 400
criterion = nn.BCELoss()
neuralnet = NeuralNet(emb_size=emb_size).to(device)
optimizer = torch.optim.Adam(neuralnet.parameters(), lr=0.0001, weight_decay=0.0000001)
trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
running_loss = 0
for epoch in range(max_epochs):
    for batch in trainloader:
        x_batch = batch[:, 0:-1].to(device)
        y_batch = torch.reshape(batch[:, -1], (batch.shape[0], 1)).to(device)
        if len(y_batch) != batch_size:
            continue
        optimizer.zero_grad()
        estimate = neuralnet.forward(x_batch.clone().detach().requires_grad_(True).float())
        loss = criterion(estimate, y_batch.clone().detach().requires_grad_(True).float())
        loss.backward()
        optimizer.step()
        loss = loss.item()
        running_loss += loss
    if epoch % epoch_print_num == 0 and epoch > 0:
        ave_loss = round(running_loss / (epoch_print_num * (train.shape[0] / batch_size)), 6)
        print(f'Average loss per text at epoch number {epoch} : {ave_loss}')
        running_loss = 0

Average loss per text at epoch number 5 : 0.456311
Average loss per text at epoch number 10 : 0.32582
Average loss per text at epoch number 15 : 0.308023
Average loss per text at epoch number 20 : 0.294298
Average loss per text at epoch number 25 : 0.2822


In [8]:
# test on test set

testloader = torch.utils.data.DataLoader(test, batch_size=test.shape[0], shuffle=False)
with torch.no_grad():
    for batch in testloader:
        x_batch = batch[:, 0:-1].to(device)
        y_batch = batch[:, -1]
        estimate = np.array(neuralnet.eval().forward(x_batch.clone().detach().float()).cpu())
accuracy = accuracy_score(y_batch, np.around(estimate, 0))
print(f'accuracy on test set: {round(accuracy, 2)}')

accuracy on test set: 0.85
